# 采集百度地图的地铁数据
1. 打开本目录下的baidu.html
2. 将下载的CityList.json文件复制到本目录下
3. 浏览器中baidu.html页面，检查-网络-双击以“?qt=subways&”开头的项目，复制url为query_url_raw，如下
query_url_raw = "https://api.map.baidu.com/?qt=subways&c=75&format=json&ak=131b9959f097765f0de45f6b1767e776&v=3.0&from=jsapi&type=subway&callback=BMapSub._rd._cbk28255"c
4. 遍历city_code_list，将query_url_raw中的c修改为城市代码，即可请求每个城市的数据

In [15]:
import json
import requests
from collections import defaultdict
import time, random

import pandas as pd

In [62]:
import sys
sys.path.append("../../..")
from secure.db_account import SubwayPrd
from k_libs.db_query import DBOperate

## 配置项

In [2]:
# 采集百度地图的地铁数据
query_url_raw = "https://api.map.baidu.com/?qt=subways&c=75&format=json&ak=131b9959f097765f0de45f6b1767e776&v=3.0&from=jsapi&type=subway&callback=BMapSub._rd._cbk28255"



In [3]:
crawler_id = 250928

## 城市列表

In [70]:
# 获取cityList
with open("cityList.json", "r", encoding="utf-8") as f:
    city_list = json.load(f)
city_code_list = [city["citycode"] for city in city_list]
city_name_list = [city["name"] for city in city_list]
city_name_e_list = [city["keyword"] for city in city_list]

In [81]:
city_name_list

['北京',
 '上海',
 '广州',
 '深圳',
 '重庆',
 '天津',
 '石家庄',
 '南京',
 '成都',
 '沈阳',
 '杭州',
 '武汉',
 '长沙',
 '苏州',
 '大连',
 '长春',
 '西安',
 '昆明',
 '佛山',
 '哈尔滨',
 '郑州',
 '宁波',
 '无锡',
 '温州',
 '青岛',
 '南昌',
 '福州',
 '东莞',
 '南宁',
 '合肥',
 '厦门',
 '乌鲁木齐',
 '贵阳',
 '香港特别行政区',
 '台北',
 '高雄',
 '南通',
 '资阳',
 '鄂州',
 '湘潭',
 '咸阳',
 '许昌',
 '常州',
 '清远',
 '济南',
 '兰州',
 '徐州',
 '呼和浩特',
 '太原',
 '金华',
 '洛阳',
 '绍兴',
 '芜湖',
 '台州',
 '滁州',
 '湘西土家族苗族自治州',
 '澳门特别行政区',
 '新加坡',
 '曼谷',
 '名古屋',
 '横滨',
 '京都',
 '大阪',
 '东京',
 '釜山',
 '大邱',
 '大田',
 '光州',
 '首尔',
 '巴黎',
 '里约热内卢',
 '巴西利亚',
 '圣保罗',
 '纽约',
 '莫斯科',
 '鹿特丹',
 '伊斯坦布尔',
 '巴塞罗那',
 '马德里']

In [80]:
len(city_code_list)

79

## 保存到本地json

In [5]:
subway_data = []
for city_code, city_name, city_name_e in zip(city_code_list, city_name_list, city_name_e_list):
    print(f"正在采集城市 {city_code}:{city_name} 的地铁数据")
    query_url = query_url_raw.replace("c=75", f"c={city_code}")
    res = requests.get(query_url)
    # 提取res中的json部分
    res_text = res.text
    res_json_str = res_text[res_text.index("{") : res_text.rindex("}") + 1]
    res_json = json.loads(res_json_str)
    city_data = {
        'city_code': city_code,
        'city_name': city_name,
        'city_name_e': city_name_e,
        'city_data': res_json['subways'],
        'crawler_id': crawler_id

    }
    subway_data.append(city_data)
    time.sleep(random.uniform(1, 3))  # 避免请求过快

正在采集城市 131:北京 的地铁数据
正在采集城市 289:上海 的地铁数据
正在采集城市 257:广州 的地铁数据
正在采集城市 340:深圳 的地铁数据
正在采集城市 132:重庆 的地铁数据
正在采集城市 332:天津 的地铁数据
正在采集城市 150:石家庄 的地铁数据
正在采集城市 315:南京 的地铁数据
正在采集城市 75:成都 的地铁数据
正在采集城市 58:沈阳 的地铁数据
正在采集城市 179:杭州 的地铁数据
正在采集城市 218:武汉 的地铁数据
正在采集城市 158:长沙 的地铁数据
正在采集城市 224:苏州 的地铁数据
正在采集城市 167:大连 的地铁数据
正在采集城市 53:长春 的地铁数据
正在采集城市 233:西安 的地铁数据
正在采集城市 104:昆明 的地铁数据
正在采集城市 138:佛山 的地铁数据
正在采集城市 48:哈尔滨 的地铁数据
正在采集城市 268:郑州 的地铁数据
正在采集城市 180:宁波 的地铁数据
正在采集城市 317:无锡 的地铁数据
正在采集城市 178:温州 的地铁数据
正在采集城市 236:青岛 的地铁数据
正在采集城市 163:南昌 的地铁数据
正在采集城市 300:福州 的地铁数据
正在采集城市 119:东莞 的地铁数据
正在采集城市 261:南宁 的地铁数据
正在采集城市 127:合肥 的地铁数据
正在采集城市 194:厦门 的地铁数据
正在采集城市 92:乌鲁木齐 的地铁数据
正在采集城市 146:贵阳 的地铁数据
正在采集城市 2912:香港特别行政区 的地铁数据
正在采集城市 9002:台北 的地铁数据
正在采集城市 9019:高雄 的地铁数据
正在采集城市 161:南通 的地铁数据
正在采集城市 242:资阳 的地铁数据
正在采集城市 122:鄂州 的地铁数据
正在采集城市 313:湘潭 的地铁数据
正在采集城市 323:咸阳 的地铁数据
正在采集城市 155:许昌 的地铁数据
正在采集城市 348:常州 的地铁数据
正在采集城市 197:清远 的地铁数据
正在采集城市 288:济南 的地铁数据
正在采集城市 36:兰州 的地铁数据
正在采集城市 316:徐州 的地铁数据
正在采集城市 321:呼和浩特 的地铁数据
正在采集城市 176:太原 的地铁数据
正在采集城市 333:金

In [ ]:
# 将subway_data保存为json文件
with open(f"baidu_subway_data_{str(crawler_id)}.json", "w", encoding="utf-8") as f:
    json.dump(subway_data, f, ensure_ascii=False, indent=2)

## 保存到数据库

In [71]:
line_fields = ['lid', 'lb', 'slb', 'n', 'loop', 'lbx', 'lby', 'lbr', 'lc', 'uid', 'uid2']
station_fields = ['sid', 'lb', 'x', 'y', 'rx', 'ry', 'st', 'ex', 'iu', 'rc', 'slb', 'ln', 'int', 'uid', 'px', 'py']
DBO = DBOperate(SubwayPrd)

In [ ]:
for city_code, city_name, city_name_e in zip(city_code_list, city_name_list, city_name_e_list):
    print(f"正在采集城市 {city_code}:{city_name} 的地铁数据")
    query_url = query_url_raw.replace("c=75", f"c={city_code}")
    res = requests.get(query_url)
    # 提取res中的json部分
    res_text = res.text
    res_json_str = res_text[res_text.index("{") : res_text.rindex("}") + 1]
    res_json = json.loads(res_json_str)
    city_data = res_json['subways']
    city_dict = defaultdict(list)
    for line in city_data['l']:
        line_info = line['l_xmlattr']
        for st in line['p']:
            st_info = st['p_xmlattr']
            # 线路数据
            for i in line_fields:
                line_i = f'line_{i}'
                city_dict[line_i].append(str(line_info.get(i, None)))
            # 站点数据
            for j in station_fields:
                station_j = f'st_{j}'
                city_dict[station_j].append(str(st_info.get(j, None)))
    city_df = pd.DataFrame.from_dict(city_dict)
    city_df['city_id'] = city_code
    city_df['city_name'] = city_name
    city_df['city_name_e'] = city_name_e
    city_df['crawler_id'] = crawler_id
    city_df['crawler_date'] = time.strftime("%Y-%m-%d", time.localtime())
    city_df['uid'] = city_df['city_id'] + city_df['crawler_id'].astype(str) + city_df.index.astype(str)
    DBO.df_to_sql(city_df, "ods_subway_baidu", if_exists="append")
    time.sleep(random.uniform(1, 5))  # 避免请求过快

正在采集城市 131:北京 的地铁数据


In [86]:
city_df

,line_lid,line_lb,line_slb,line_n,line_loop,line_lbx,line_lby,line_lbr,line_lc,line_uid,...,st_int,st_uid,st_px,st_py,city_id,city_name,city_name_e,crawler_id,crawler_date,uid
0,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,291c5802f26a751cbca240d9,12935140.04,4825694.5,131,北京,beijing,250928,2025-09-28,1312509280
1,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,ad28546df35285eb851541d9,12937624.6,4825645.68,131,北京,beijing,250928,2025-09-28,1312509281
2,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,a047555503a5bc5cbc4842d9,12940185.58,4825661.64,131,北京,beijing,250928,2025-09-28,1312509282
3,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,a27fa5a23a128501177643d9,12942086.4,4825707.12,131,北京,beijing,250928,2025-09-28,1312509283
4,地铁1号线八通线,1号线八通线,False,23,False,-498.9,139.1,0,0xc03935,bce557d6f7fadd4ea5da39b7,...,3,53889c15034f2e3f58bbbcde,12944444.84,4825755.15,131,北京,beijing,250928,2025-09-28,1312509284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,地铁亦庄T1线,亦庄T1线,True,9,False,255,867,-25,0xe4071c,4d0cb5d7f4758c94a80dd871,...,2,ef30d7917b32ccf2c51b7589,12970087.47,4802505.23,131,北京,beijing,250928,2025-09-28,131250928597
598,地铁亦庄T1线,亦庄T1线,True,9,False,255,867,-25,0xe4071c,4d0cb5d7f4758c94a80dd871,...,2,9f5957cfd2e00071e9f85338,12969782.95,4801703.89,131,北京,beijing,250928,2025-09-28,131250928598
599,地铁亦庄T1线,亦庄T1线,True,9,False,255,867,-25,0xe4071c,4d0cb5d7f4758c94a80dd871,...,2,2ce4ae766cbd7a6c03ec01c1,12969964.31,4800069.42,131,北京,beijing,250928,2025-09-28,131250928599
600,地铁亦庄T1线,亦庄T1线,True,9,False,255,867,-25,0xe4071c,4d0cb5d7f4758c94a80dd871,...,2,2af8a6bffeb46b755bf6be44,12970161.47,4799033.04,131,北京,beijing,250928,2025-09-28,131250928600
